In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in

import sys
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # plotting
import seaborn as sns
%matplotlib inline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
import kagglegym

env = kagglegym.make()
observation = env.reset()

In [ ]:
train = observation.train
ID = "id"
TSTAMP = "timestamp"
TARGET = "y"
(len(train["timestamp"].unique()), min(train["timestamp"]), max(train["timestamp"]))

In [ ]:
def findMissingCounts(df, columns):
    labels = []
    missing = []
    for column in columns:
        labels.append(column)
        missing.append(df[column].isnull().sum())
    return labels, missing

In [ ]:
def createHorizontalBarPlot(labels, missing, plot_width, plot_height):
    N = len(labels)
    ind = np.arange(N)
    width = 0.35

    fig, ax = plt.subplots(figsize = (plot_width, plot_height))
    rects = ax.barh(ind, missing, width, align='center', color="lightskyblue")
    ax.set_yticks(ind + width)
    ax.set_yticklabels(labels)
    plt.show()
    plt.close()

In [ ]:
def findMatchedColumnsUsingPrefix(prefix, df):
    columns = df.columns[df.columns.str.startswith(prefix)]
    return list(columns.values)
    
derived_columns = findMatchedColumnsUsingPrefix("derived", train)
fundamental_columns = findMatchedColumnsUsingPrefix("fundamental", train)
technical_columns = findMatchedColumnsUsingPrefix("technical", train)

In [ ]:
ids = train[ID].unique()
tstamps = train[TSTAMP].unique()

In [ ]:
id_groups = train.groupby(ID)
sample_df = id_groups.get_group(ids[0]).reset_index(drop = True)
sample_df.head()

In [ ]:
def findMissingEntriesForIds(columns, train):
    indexes = []
    rows = {}
    for id, group in train.groupby('id'):
        indexes.append(id)
        for column in columns:
            rows[column] = sample_df[column].isnull().sum()
            
    df = pd.DataFrame(rows, index = indexes)
    df.columns = pd.MultiIndex.from_tuples([tuple(c.split('_')) for c in df.columns])
    df = df.stack(0).reset_index(1)
    df.sort_index()
    return df

In [ ]:
result = findMissingEntriesForIds(derived_columns, train)
result.head(2)

In [ ]:
result = findMissingEntriesForIds(fundamental_columns, train)
result.head(2)

In [ ]:
result = findMissingEntriesForIds(technical_columns, train)
result.head(2)

In [ ]:
def findMissingEntriesForIds(columns, train):
    indexes = []
    rows = {}
    for id, group in train.groupby('id'):
        indexes.append(id)
        for column in columns:
            rows[column] = sample_df[column].isnull().sum()
            
    df = pd.DataFrame(rows, index = indexes)
    df.columns = pd.MultiIndex.from_tuples([tuple(c.split('_')) for c in df.columns])
    df = df.stack(0).reset_index(1)
    df.sort_index()
    return df

In [ ]:
def findMaximumNullColumns(res):
    names = []
    values = []
    for column in res.columns:
        if column == 'level_1':
            continue
        if res[column].sum() > 0:
            name = res['level_1'].unique()[0] + '_' + column
            names.append(name)
            values.append(res[column].sum())
    return pd.DataFrame({'columns' : names, 'counts' : values})

In [ ]:
res = findMaximumNullColumns(findMissingEntriesForIds(derived_columns, train))
labels = res['columns']
missing = res['counts']
createHorizontalBarPlot(labels, missing, 5, 5)

In [ ]:
res = findMaximumNullColumns(findMissingEntriesForIds(fundamental_columns, train))
labels = res['columns']
missing = res['counts']
createHorizontalBarPlot(labels, missing, 5, 5)

In [ ]:
res = findMaximumNullColumns(findMissingEntriesForIds(technical_columns, train))
labels = res['columns']
missing = res['counts']
createHorizontalBarPlot(labels, missing, 5, 5)